In [3]:
import pandas as pd
import numpy as np

In [4]:
#Reading user file
u_cols = ['user_id','age', 'sex' ,'occupation' , 'zipcode']
users=pd.read_csv('u.user',sep='|',names=u_cols,encoding='latin-1')

In [5]:
users.head()

,user_id,age,sex,occupation,zipcode
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [6]:
#Reading ratings file
r_cols = ['user_id','movie_id','rating','Timestamp']
ratings = pd.read_csv('u.data',sep='\t',names=r_cols,encoding='latin-1')

In [7]:
ratings.head()

,user_id,movie_id,rating,Timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [8]:
#Reading items file
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('u.item',sep='|',names=i_cols,encoding='latin-1')

In [9]:
items.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [10]:
users.shape


(943, 5)

In [11]:
ratings.shape

(100000, 4)

In [13]:
r_cols = ['user_id','movie_id','rating','Timestamp']
ratings_train = pd.read_csv('ua.base',sep='\t',names=r_cols,encoding='latin-1')
ratings_test = pd.read_csv('ua.test',sep='\t',names=r_cols,encoding='latin-1')

In [14]:
ratings_train.shape

(90570, 4)

In [15]:
ratings_train.head()

,user_id,movie_id,rating,Timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [16]:
ratings_test.head()

,user_id,movie_id,rating,Timestamp
0,1,20,4,887431883
1,1,33,4,878542699
2,1,61,4,878542420
3,1,117,3,874965739
4,1,155,2,878542201


In [17]:
ratings_test.shape

(9430, 4)

In [18]:
# We will recommend movies based on user-user similarity and item-item similarity. 
# For that, first we need to calculate the number of unique users and movies.
n_users = ratings.user_id.unique().shape[0]
n_movies = ratings.movie_id.unique().shape[0]

In [19]:
n_users

943

In [20]:
n_movies

1682

In [21]:
# Here we will create user-item matrix to calculate similarity between user and item
data_matrix = np.zeros((n_users,n_movies))

In [22]:
data_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
for lines in ratings.itertuples() :
    data_matrix[lines[1]-1,lines[2]-1]=lines[3]

In [25]:
#we will calculate the similarity. We can use the pairwise_distance function from sklearn 
# to calculate the cosine similarity
from sklearn.metrics.pairwise import pairwise_distances

In [26]:
user_similarity = pairwise_distances(data_matrix,metric='cosine')

In [27]:
item_similarity = pairwise_distances(data_matrix.T,metric='cosine')

In [28]:
user_similarity[1][2]

0.8894086753916608

In [29]:
# This gives us the item-item and user-user similarity in an array form. 
# The next step is to make predictions based on these similarities. 
# Let’s define a function to do just that.
def predict(ratings,similarity, type='user'):
    if type == 'user' :
        mean_user_rating = ratings.mean(axis=1)
        ratings_diff = (ratings - mean_user_rating[:,np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
        
    return pred


In [30]:
#finally we can generate prediction based on user similarity and item similarity
user_prediction = predict(data_matrix, user_similarity, type='user')
item_prediction = predict(data_matrix, item_similarity, type='item')

In [31]:
user_prediction

array([[ 2.06532606,  0.73430275,  0.62992381, ...,  0.39359041,
         0.39304874,  0.3927712 ],
       [ 1.76308836,  0.38404019,  0.19617889, ..., -0.08837789,
        -0.0869183 , -0.08671183],
       [ 1.79590398,  0.32904733,  0.15882885, ..., -0.13699223,
        -0.13496852, -0.13476488],
       ...,
       [ 1.59151513,  0.27526889,  0.10219534, ..., -0.16735162,
        -0.16657451, -0.16641377],
       [ 1.81036267,  0.40479877,  0.27545013, ..., -0.00907358,
        -0.00846587, -0.00804858],
       [ 1.8384313 ,  0.47964837,  0.38496292, ...,  0.14686675,
         0.14629808,  0.14641455]])

In [32]:
# We can also use collaborative filltering model using ' Turicreate ' for recommendation 
# but it is applicable only on mac-os and linux-os

In [33]:
# Movie Recommendation using matrix factorization method
# Here I will use SVD(singular value Decomposition ) algo to decompose rating matrix 

In [34]:
#Here we have rating df
ratings.head()

,user_id,movie_id,rating,Timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [37]:
R_df=ratings.pivot(index='user_id',columns='movie_id',values='rating').fillna(0)

In [47]:
R_df.shape

(943, 1682)

In [39]:
#Here we convert this R_df dataframe to numpy array
R=R_df.as_matrix()

C:\Users\jai\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [40]:
R

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [41]:
# This is mean array of user's ratings
user_ratings_mean=np.mean(R,axis=1)

In [45]:
user_ratings_mean.reshape(-1,1).shape

(943, 1)

In [49]:
R_demeaned = R-user_ratings_mean.reshape(-1,1)

In [51]:
#R_demeaned

array([[ 4.41617122,  2.41617122,  3.41617122, ..., -0.58382878,
        -0.58382878, -0.58382878],
       [ 3.86325803, -0.13674197, -0.13674197, ..., -0.13674197,
        -0.13674197, -0.13674197],
       [-0.08977408, -0.08977408, -0.08977408, ..., -0.08977408,
        -0.08977408, -0.08977408],
       ...,
       [ 4.9470868 , -0.0529132 , -0.0529132 , ..., -0.0529132 ,
        -0.0529132 , -0.0529132 ],
       [-0.20035672, -0.20035672, -0.20035672, ..., -0.20035672,
        -0.20035672, -0.20035672],
       [-0.34066587,  4.65933413, -0.34066587, ..., -0.34066587,
        -0.34066587, -0.34066587]])

In [ ]:
# Here I’m going to use the Scipy function svds because 
#it let’s me choose how many latent factors I want to use to approximate the original ratings matrix

In [52]:
from scipy.sparse.linalg import svds

In [53]:
U , sigma , Vt = svds(R_demeaned,k=50) # Here k is no. of latent feature

In [65]:
sigma=np.diag(sigma)

In [72]:
#U

array([[ 0.13944814,  0.08802883,  0.11959544, ..., -0.00473136,
         0.0043773 , -0.06653149],
       [ 0.02104373,  0.03419113,  0.00195072, ...,  0.05392487,
        -0.04620115, -0.01309312],
       [-0.01493341,  0.00562006, -0.02046879, ...,  0.02317463,
        -0.02481712, -0.00320484],
       ...,
       [ 0.01119652, -0.00552595,  0.00069347, ...,  0.00746046,
        -0.02554262, -0.0082399 ],
       [ 0.05682848,  0.00132044,  0.08516041, ...,  0.02379019,
         0.00759561, -0.02504761],
       [ 0.00657694,  0.02726909,  0.06758361, ..., -0.05701743,
        -0.01320454, -0.04472769]])

In [66]:
sigma.shape

(50, 50)

In [60]:
Vt.shape

(50, 1682)

In [71]:
all_user_predicted_ratings = np.dot(np.dot(U,sigma),Vt)+user_ratings_mean.reshape(-1,1)

In [73]:
pred_df=pd.DataFrame(all_user_predicted_ratings,columns=R_df.columns)

In [74]:
pred_df.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,6.488436,2.959503,1.634987,3.024467,1.656526,1.659506,3.630469,0.240669,1.791518,3.347816,...,0.011976,-0.092017,-0.074553,-0.060985,0.009427,-0.035641,-0.039227,-0.037434,-0.025552,0.023513
1,2.347262,0.129689,-0.098917,0.328828,0.159517,0.481361,0.213002,0.097908,1.892100,0.671000,...,0.003943,-0.026939,-0.035460,-0.029883,-0.027153,-0.015244,-0.008277,-0.011760,0.011639,-0.046924
2,0.291905,-0.263830,-0.151454,-0.179289,0.013462,-0.088309,-0.057624,0.568764,-0.018506,0.280742,...,-0.028964,-0.031622,0.045513,0.026089,-0.021705,0.002282,0.032363,0.017322,-0.006644,-0.009480
3,0.366410,-0.443535,0.041151,-0.007616,0.055373,-0.080352,0.299015,-0.010882,-0.160888,-0.118834,...,0.020069,0.015981,-0.000182,0.005593,0.026634,0.023562,0.036405,0.029984,0.015612,-0.008713
4,4.263488,1.937122,0.052529,1.049350,0.652765,0.002836,1.730461,0.870584,0.341027,0.569055,...,0.019973,-0.053521,-0.017242,-0.007137,-0.038987,0.010338,0.004869,0.007603,-0.020575,0.003330


In [136]:
movies_df=items.rename(columns={'movie id':'movie_id'})

In [138]:
items.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [135]:
items.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [145]:

# Now using this function we will recommend the movie for a user
def recommend_movie(prediction_df,userID,movies_df,original_ratings_df,num_recommendation=5):
    user_row_number = userID-1 #Because userId start with 1 and index start with 0
    
    sorted_user_predictions = prediction_df.iloc[user_row_number].sort_values(ascending=False)
    
    user_data = original_ratings_df[original_ratings_df.user_id == (userID)]
#     print(user_data)
#     print(movies_df)
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'movie_id', right_on = 'movie_id').
                     sort_values(['rating'], ascending=False)
                 )
    #print(user_full)
    print('Recommending the highest {0} predicted ratings movies not already rated.'.format(num_recommendation))
    
#     #Recommend the highest predicted rating movies that user has'nt watched
#     #print(user_full['movie_id'])
 #   print(sorted_user_predictions)
    recommendations = (movies_df[~movies_df['movie_id'].isin(user_full['movie_id'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movie_id',
               right_on = 'movie_id').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendation, :-1]
                      )

    return user_full, recommendations
already_watched,recommended_movies = recommend_movie(pred_df,837,movies_df,ratings,10)
    

Recommending the highest 10 predicted ratings movies not already rated.


In [147]:
already_watched

,user_id,movie_id,rating,Timestamp,movie title,release date,video release date,IMDb URL,unknown,Action,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
44,837,740,5,875722123,Jane Eyre (1996),05-Apr-1996,NaN,http://us.imdb.com/M/title-exact?Jane%20Eyre%2...,0,0,...,0,0,0,0,0,1,0,0,0,0
37,837,1009,5,875721765,Stealing Beauty (1996),14-Jun-1996,NaN,http://us.imdb.com/M/title-exact?Stealing%20Be...,0,0,...,0,0,0,0,0,0,0,0,0,0
9,837,283,5,875722069,Emma (1996),02-Aug-1996,NaN,http://us.imdb.com/M/title-exact?Emma%20(1996),0,0,...,0,0,0,0,0,1,0,0,0,0
36,837,125,5,875722032,Phenomenon (1996),29-Jun-1996,NaN,http://us.imdb.com/M/title-exact?Phenomenon%20...,0,0,...,0,0,0,0,0,1,0,0,0,0
11,837,289,5,875721539,Evita (1996),25-Dec-1996,NaN,http://us.imdb.com/M/title-exact?Evita%20(1996),0,0,...,0,0,0,1,0,0,0,0,0,0
24,837,151,5,875721734,Willy Wonka and the Chocolate Factory (1971),01-Jan-1971,NaN,http://us.imdb.com/M/title-exact?Willy%20Wonka...,0,0,...,0,0,0,0,0,0,0,0,0,0
23,837,258,4,875721473,Contact (1997),11-Jul-1997,NaN,http://us.imdb.com/Title?Contact+(1997/I),0,0,...,0,0,0,0,0,0,1,0,0,0
14,837,20,4,875721919,Angels and Insects (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Angels%20and%...,0,0,...,0,0,0,0,0,1,0,0,0,0
22,837,294,4,875721502,Liar Liar (1997),21-Mar-1997,NaN,http://us.imdb.com/Title?Liar+Liar+(1997),0,0,...,0,0,0,0,0,0,0,0,0,0
27,837,328,4,875721604,Conspiracy Theory (1997),08-Aug-1997,NaN,http://us.imdb.com/M/title-exact?Conspiracy+Th...,0,1,...,0,0,0,0,1,1,0,1,0,0


In [146]:
recommended_movies

,movie_id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
11,14,"Postino, Il (1994)",01-Jan-1994,NaN,"http://us.imdb.com/M/title-exact?Postino,%20Il...",0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
107,116,Cold Comfort Farm (1995),23-Apr-1996,NaN,http://us.imdb.com/M/title-exact?Cold%20Comfor...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
116,126,"Spitfire Grill, The (1996)",06-Sep-1996,NaN,http://us.imdb.com/M/title-exact?Spitfire%20Gr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42,50,Star Wars (1977),01-Jan-1977,NaN,http://us.imdb.com/M/title-exact?Star%20Wars%2...,0,1,1,0,0,...,0,0,0,0,0,1,1,0,1,0
238,255,My Best Friend's Wedding (1997),20-Jun-1997,NaN,http://us.imdb.com/M/title-exact?My+Best+Frien...,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
440,471,Courage Under Fire (1996),08-Mar-1996,NaN,http://us.imdb.com/M/title-exact?Courage%20Und...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
92,100,Fargo (1996),14-Feb-1997,NaN,http://us.imdb.com/M/title-exact?Fargo%20(1996),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
258,282,"Time to Kill, A (1996)",13-Jul-1996,NaN,http://us.imdb.com/M/title-exact?Time%20to%20K...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
704,742,Ransom (1996),08-Nov-1996,NaN,http://us.imdb.com/M/title-exact?Ransom%20(1996),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [125]:
items.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


,movie_id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
items.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [81]:
ratings.head() 

,user_id,movie_id,rating,Timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
